#### **Goal:**

Upload PDFs (earnings reports, trading logs, research papers) → AI answers questions.

**Reinforces:**

* LangChain ingestion pipeline

* Text splitting

* Embeddings + vector DB

* Prompt engineering

* Streamlit UI.

**Workflow:**

1. Upload PDF/CSV.

2. Chunk text with splitter.

3. Create embeddings + store in vector DB.

4. Retrieval + LLM answering.

5. Streamlit chat interface

#### **Extra challenge:**

Add summary + risk insight generation.

## Step 1 Financial PDFs:

Use:

Company annual reports

Trading journals (export from MT5 if possible)

Research papers

Financial news PDFs.

If you want quick test data:

Kaggle financial datasets

Yahoo Finance CSV exports

SEC filings.

In [1]:
# !pip install pdfminer
# !pip install pdfplumber

In [2]:
from langchain_community.document_loaders import PyMuPDFLoader, CSVLoader, TextLoader, UnstructuredPDFLoader
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings, ChatNVIDIA

In [3]:
file = './data/Nestle-India-Annual-Report-2020.pdf'

In [4]:
import pdfplumber

with pdfplumber.open(file) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        tables = page.extract_tables()


In [6]:
loader = UnstructuredPDFLoader(file_path=file)
data1=loader.lazy_load()
data2= loader.alazy_load()

In [7]:
print(f"{data1}\n{data2}")

<generator object UnstructuredBaseLoader.lazy_load at 0x7b9c96d5d070>
<async_generator object BaseLoader.alazy_load at 0x7b9c96d53cc0>


## Data Ingestion
Load PDFs → Extract text → Clean text

**Use:**

PDF loader

Text preprocessing.

In [15]:
if file[-3:]=="pdf":
    loader = PyMuPDFLoader(file_path=file)
elif file[-3:]=="txt":
    loader = TextLoader(file_path=file)
else:
    loader = CSVLoader(file_path=file)
    
data =loader.load()
data[0]

Document(metadata={'producer': 'Adobe PDF library 15.00', 'creator': 'Adobe Illustrator 25.2 (Windows)', 'creationdate': '2021-04-12T20:06:52+06:30', 'source': 'Nestle-India-Annual-Report-2020.pdf', 'file_path': 'Nestle-India-Annual-Report-2020.pdf', 'total_pages': 136, 'format': 'PDF 1.6', 'title': 'PART 1', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2021-04-12T22:02:18+05:30', 'trapped': '', 'modDate': "D:20210412220218+05'30'", 'creationDate': "D:20210412200652+06'30'", 'page': 0}, page_content='Nestlé India Limited | Annual Report - 2020\nMarching Together\n#AsAForceForGood')

### STEP 3 — Text Splitting

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [17]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=100)
spliter  = text_splitter.split_documents(data)
spliter[0]

Document(metadata={'producer': 'Adobe PDF library 15.00', 'creator': 'Adobe Illustrator 25.2 (Windows)', 'creationdate': '2021-04-12T20:06:52+06:30', 'source': 'Nestle-India-Annual-Report-2020.pdf', 'file_path': 'Nestle-India-Annual-Report-2020.pdf', 'total_pages': 136, 'format': 'PDF 1.6', 'title': 'PART 1', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2021-04-12T22:02:18+05:30', 'trapped': '', 'modDate': "D:20210412220218+05'30'", 'creationDate': "D:20210412200652+06'30'", 'page': 0}, page_content='Nestlé India Limited | Annual Report - 2020\nMarching Together\n#AsAForceForGood')

### STEP 4 — Embeddings + Vector DB
STEP 4 — Embeddings + Vector DB
Vector DB options:

FAISS (simple local)

ChromaDB.

In [18]:
import os
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
import chromadb
load_dotenv()

True

In [19]:
os.environ['NVIDIA_API_KEY'] = os.getenv('NVIDIA_API_KEY')
api_key=os.getenv('NVIDIA_API_KEY')

In [29]:
Embedding = NVIDIAEmbeddings(nvidia_api_key=api_key)
llm = ChatNVIDIA(model='nvidia/nemotron-3-nano-30b-a3b',nvidia_api_key=api_key)


In [21]:
vectorstore= FAISS.from_documents(documents=spliter,embedding=Embedding)
vectorstore

In [22]:
quary = "give sort summery on REPORT ON CORPORATE GOVERNANCE FOR THE YEAR ENDED DECEMBE"
result= vectorstore.similarity_search(query=quary)
result[0].page_content

"Marching\nTogether\n92. Annexure 1: Report on Corporate Governance\n107. Annexure 2: Annual Report on (CSR) Activities \n115. Annexure 3: Business Responsibility Report\n124. Annexure 4: Secretarial Audit Report\n127. Annexure 5: Report on Conservation of Energy etc.\n130. Annexure 6: Information Regarding Employees\n \n \n \nand Related Disclosures\nAnnexures to the Board’s Report\nIndex\n01. Message to Shareholders\n03. Purpose\n06. Planet\n09. Partnerships\n12. People\n15. Performance\n18. Corporate Information\n19. 10 - Year Financial Highlights\n20. Board's Report\n36. Auditor’s Report\n43. Annual Accounts\n131. Dividend Distribution Policy"

### STEP 5 — Retrieval Pipeline (RAG)
User query

Vector similarity search

Relevant chunks retrieved

LLM generates answer.

In [23]:
## Retriever chain, Document chain

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, chain
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    
    """You are a financial analysis assistant. 
    Answer strictly from provided context.
    If information is missing, say "Not found in document."

    FORMAT RULES:
    - Use clear headings
    - Use bullet points or short paragraphs
    - Avoid unnecessary tables unless explicitly requested
    - Keep answers concise and readable
    - No decorative markdown like **bold everywhere**
    Context:
    {retrieved_text}
    
    Question:
    {user_question}
    """
)

In [ ]:
retrive= vectorstore.as_retriever(search_kwargs={"k": 5},search_type="mmr")


In [25]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [26]:
from langchain_core.runnables import RunnableLambda

chain = (
    {
        "retrieved_text": retrive| RunnableLambda(format_docs),
        "user_question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [27]:
# chain = (
#     {
#         "retrieved_text": retrive,
#         "user_question": RunnablePassthrough()
 
        
#     }
#     | prompt
#     | llm
#     | StrOutputParser()
# )
# chain

In [28]:
response = chain.invoke("what is NESTLÉ’S PHILOSOPHY ?")
print(response)


**Nestlé’s Philosophy**

- **Core Purpose**  
  - “We unlock the power of food to enhance quality of life for everyone, today and for generations to come.”

- **Guiding Ambitions**  
  1. Enable healthier and happier lives for individuals and families.  
  2. Help develop thriving and resilient communities.  
  3. Steward the planet’s natural resources for future generations.

- **Foundational Principles & Values**  
  - Rooted in a robust set of principles and values based on respect.  

- **Approach to Stakeholders**  
  - Engages farmers, women, mothers, suppliers, and the broader community to create positive social, economic, and environmental impact.


### Future update
Better PDF Extraction

* import pdfplumber

with pdfplumber.open(file) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        tables = page.extract_tables()

* STEP 2 — Numeric Normalization
import re

def normalize_numbers(text):
    text = re.sub(r'(\d+)%', r'\1 percent', text)
    text = re.sub(r'\$(\d+\.?\d*)B', r'\1 billion dollars', text)
    return text

* STEP 4 — Hybrid Storage
Store:

Text chunks → Vector DB (FAISS)

Tables → JSON file or PostgreSQL

Example:

import json

with open("tables.json", "w") as f:
    json.dump(all_tables, f)

STEP 5 — Hybrid Retrieval Logic

When user asks:

Check if query contains numeric keywords:

growth

revenue

percentage

profit

rate

If yes:
→ Search structured tables first.

Then:
→ Add semantic vector retrieval context.

Combine both into final prompt.

### STEP 6 — Numeric-Aware Prompt

You are a financial analysis assistant.

Use BOTH:
1. Retrieved text context
2. Structured financial table data

Important:
- Preserve all numbers exactly.
- Include growth rates and percentages.
- Do not estimate.

Context:
{retrieved_text}

Table Data:
{structured_data}

Question:
{user_question}

### STEP 7 — Streamlit Interface
Simple UI:

File upload

Chat input

Response display

Optional summary button.

Focus on usability.

### Project desc

---

##### 📌 Short GitHub Project Description (For Repo Summary)

Use this in your GitHub description box:

> Hybrid RAG-based Financial AI Assistant built with LangChain & Streamlit. Preserves financial metrics, growth rates, and structured data using semantic + table-aware retrieval.

---

# ⭐ Optional: LinkedIn Portfolio Description

If you want to post about it:

> Built a Financial AI Assistant using Hybrid RAG architecture that preserves structured financial metrics (growth %, revenue, KPIs) by combining vector search with table-aware extraction. Designed for accurate financial document intelligence and scalable AI deployment.


#### 1. Financial Insight Prompt
Analyze this financial text.
Give:
- Key insights
- Risks
- Opportunities
- Summary in simple terms.

#### 2. Trading Journal Analysis
Analyze these trading notes.
Identify:
- Mistakes
- Risk patterns
- Improvement suggestions.

#### 3. Knowledge Base Chat Prompt
- Answer strictly from context.
- Avoid assumptions.
- Explain clearly.